***IMPORT DES DONNEES DE TRAFIC, BRUIT, METEO ET SONDAGE***

In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src\Carac_situ_acoustiq')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src')

import os, re
import pandas as pd
import numpy as np
import altair as alt
from math import pi
from IPython.display import Image, FileLink, FileLinks
from IPython.core.display import HTML 
from Import_stockage_donnees.Import_export_bdd import FichierMesureBruitCsv, FichierCsvEnquete, FichiersMeteo

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
# alt.data_transformers.disable_max_rows()
# alt.data_transformers.enable('json')

# DONNEES BRUIT

> ## Lecture des fichiers ods

In [36]:
# exemple de lecture 
test = FichierMesureBruitCsv(r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono1_10RuePierreRonsard\sono1_02-04-2022.csv')
# test.dfNiveauSpectre.head(10)

In [ ]:
# transfert vers Bdd par dossier
dossierSrc = r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon'
transfertFichierMesure2Bdd(dossierSrc)

# DONNEES RESSENTI

## Import des données brutes  
Il s'agit de lire le fichier, de le nettoyer puis de le rendre plus compréhensible.  
Comme le fichier mélange des données fournies par les riverains directement par LimeSurvey et d'autre issus du transfert depuis des questionnaires papiers, il faut également s'assurer de la présence d'un identifiant unique par riverains, en créant des adresses mail factice pour les réoponses issues de questionaire papier.

In [3]:
ressenti = FichierCsvEnquete(r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\Enquete_LimeSurvey\resultats_LimeSurvey_Bruts\resultats_complets_enquete_avec_papier.csv')

## Analyse des données de participants et déclarations  
Il s'agit de fournir des indicateurs de répartition des différentes valeurs de contexte et de déclaration fournies par les participants. Attention, les formulaires papiers ne permettaient pas de remplir tout les informations de contexte

<div class="alert alert-block alert-warning"><b>Attention:</b> Il n'y a pas d'analyse des commentaires réalisée pour le moment.</div>

In [16]:
#creation des donnees
dicoCharts = ressenti.creationDfAnalyseRetour()
# creation des charts, avec ou sans export
ressenti.creationCharts(dicoCharts, export=False)

In [ ]:
# exemple
dicoCharts['sensibilite']['data']
dicoCharts['sensibilite']['chart']

# DONNEES MÉTÉO
L'objectif est double : 
1. fournir des rose des vents selon une période de temps définie
1. fournir le statut de propagation acoustique (favorable / défavorable) selon l'angle et une période de temps définie

## Import des données brutes  
Les données sont réparties sur 2 fichiers dans le dossier suivant C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\station_meteo\datas

In [3]:
fichierMeteo = FichiersMeteo(r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\station_meteo\datas',
                             ['Mesures_Ech24_21-03-2022_04-04-2022.csv', 'Mesures_Ech24_04-04-2022_19-04-2022.csv'])

In [6]:
imageCompDirVent = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\Illustrations\meteo\ecart_dirVent_HautBas.svg'

In [4]:
dfDirVentComp = fichierMeteo.analyseDirVents(fichierMeteo.dfBrutes)

In [16]:
test = FileLink(imageCompDirVent)

[test](C:/Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\Illustrations\meteo\ecart_dirVent_HautBas.svg)